# Purge Un-Verified Images

### Ref:  camera-api/example_utils.py  
(which came from ssd-dag)  

1. use labelImg to verify
2. move xml to annotation dir
3. move jpg to jpeg_images dir
4. OPTIONAL - make tf records
5. this program will read annotation
  - move unverified annotation to trash/
  - move unverified jpeg to trash/
  
 

In [1]:
import os
import sys
import shutil

In [2]:
# append to path so it wil find the tensorflow utilities

cwd = os.getcwd()
# this path is different in this project
models = os.path.abspath(os.path.join(cwd, '..', 'models/research/'))
slim = os.path.abspath(os.path.join(cwd, '..', 'models/research/slim'))
sys.path.append(models)
sys.path.append(slim)


import gen_imagesets
import example_utils

In [3]:
## Global
DATA_DATE = '20201207'

# project directories
PROJECT = os.getcwd()
HSDATA = '/hsdata'     # high speed data

IMAGE_DIR = os.path.join(HSDATA, "jpeg_images")
ANNOTATION_DIR_ROOT = os.path.join(HSDATA, "annotation")
TRASH_DIR = os.path.join(PROJECT, "trash")
TRAINING_SPLIT_TUPLE =  (100,0,0)


In [4]:
print ("annotation root:", ANNOTATION_DIR_ROOT)
verified_list, not_verified_list = gen_imagesets.gen_verified_lists(ANNOTATION_DIR_ROOT)
print ("verified list length:", len(verified_list))
print ("non-verified list length:", len(not_verified_list))

annotation root: /hsdata/annotation
-- MAKING Verified & NonVerified IMAGES LISTS
   found 9 subdirectories
   making list of verified annotations -- verified count 17894 / non 0 in /hsdata/annotation/202007
   making list of verified annotations -- verified count 10934 / non 0 in /hsdata/annotation/202002
   making list of verified annotations -- verified count 17953 / non 5711 in /hsdata/annotation/202009
   making list of verified annotations -- verified count 11301 / non 0 in /hsdata/annotation/202001
   making list of verified annotations -- verified count 18768 / non 0 in /hsdata/annotation/202004
   making list of verified annotations -- verified count 18309 / non 0 in /hsdata/annotation/202008
   making list of verified annotations -- verified count 19473 / non 0 in /hsdata/annotation/202003
   making list of verified annotations -- verified count 19640 / non 0 in /hsdata/annotation/202005
   making list of verified annotations -- verified count 18607 / non 0 in /hsdata/annotat

In [5]:
# -- make sure you hvae a trash directory
#    camera-api/trash

# move unverfied to trash
for i, image_id in enumerate(not_verified_list):
    if image_id == '.gitkeep':
        continue
        
    xml_source, jpg_source = example_utils.get_image_annotation_paths(image_id)

    shutil.move(xml_source, TRASH_DIR)
    shutil.move(jpg_source, TRASH_DIR)


In [6]:
! ls trash | wc

  11422   11422  251284


# Backup to S3

In [7]:
tarball_name = DATA_DATE + "_annotation.tar.gz"
annotation_source = os.path.join(HSDATA, 'annotation')
! tar czvf $tarball_name $annotation_source

tar: Removing leading `/' from member names
/hsdata/annotation/
/hsdata/annotation/202007/
/hsdata/annotation/202007/15916235898-0-1-c.xml
/hsdata/annotation/202007/15913539993-1-0-c.xml
/hsdata/annotation/202007/15935450089-1-1-c.xml
/hsdata/annotation/202007/15929131140-5-2-c.xml
/hsdata/annotation/202007/15939480568-0-0-c.xml
/hsdata/annotation/202007/15936232636-1-0-c.xml
/hsdata/annotation/202007/15930871806-1-1-c.xml
/hsdata/annotation/202007/15939859411-6-0-c.xml
/hsdata/annotation/202007/15939706613-1-0-c.xml
/hsdata/annotation/202007/15930036775-1-5-c.xml
/hsdata/annotation/202007/15930123986-0-4-c.xml
/hsdata/annotation/202007/15938633560-3-0-c.xml
/hsdata/annotation/202007/15916428802-0-1-c.xml
/hsdata/annotation/202007/15929148903-0-1-c.xml
/hsdata/annotation/202007/15913077092-0-1-c.xml
/hsdata/annotation/202007/15918741109-1-1-c.xml
/hsdata/annotation/202007/15930145092-1-1-c.xml
/hsdata/annotation/202007/15918762365-3-3-c.xml
/hsdata/annotation/202007/15932683512-0-1-c.x

/hsdata/annotation/202007/15932044499-1-5-c.xml
/hsdata/annotation/202007/15918751787-0-1-c.xml
/hsdata/annotation/202007/15913153081-1-4-c.xml
/hsdata/annotation/202007/15919039251-3-0-c.xml
/hsdata/annotation/202007/15913547328-1-6-c.xml
/hsdata/annotation/202007/15916226983-0-1-c.xml
/hsdata/annotation/202007/15918269127-1-1-c.xml
/hsdata/annotation/202007/15938633692-0-0-c.xml
/hsdata/annotation/202007/15919004154-0-1-c.xml
/hsdata/annotation/202007/15918753419-1-1-c.xml
/hsdata/annotation/202007/15913768872-0-1-c.xml
/hsdata/annotation/202007/15918232216-0-1-c.xml
/hsdata/annotation/202007/15929070736-1-4-g.xml
/hsdata/annotation/202007/15935199544-0-1-c.xml
/hsdata/annotation/202007/15931118191-5-0-c.xml
/hsdata/annotation/202007/15917206777-1-1-c.xml
/hsdata/annotation/202007/15930365624-5-0-c.xml
/hsdata/annotation/202007/15928761821-0-1-g.xml
/hsdata/annotation/202007/15938956858-5-1-c.xml
/hsdata/annotation/202007/15932860158-5-1-c.xml
/hsdata/annotation/202007/15930262669-0-

/hsdata/annotation/202007/15938968609-6-1-c.xml
/hsdata/annotation/202007/15931121461-4-1-c.xml
/hsdata/annotation/202007/15929311999-0-1-c.xml
/hsdata/annotation/202007/15918759373-0-1-c.xml
/hsdata/annotation/202007/15916533774-1-4-c.xml
/hsdata/annotation/202007/15916451519-0-1-c.xml
/hsdata/annotation/202007/15930263015-0-1-c.xml
/hsdata/annotation/202007/15940670889-6-1-c.xml
/hsdata/annotation/202007/15932049658-5-2-c.xml
/hsdata/annotation/202007/15935565434-3-2-c.xml
/hsdata/annotation/202007/15918113505-1-2-c.xml
/hsdata/annotation/202007/15929279929-1-4-c.xml
/hsdata/annotation/202007/15932695189-1-0-c.xml
/hsdata/annotation/202007/15916190432-0-1-c.xml
/hsdata/annotation/202007/15913152032-0-1-c.xml
/hsdata/annotation/202007/15932044068-1-4-c.xml
/hsdata/annotation/202007/15935565877-1-0-c.xml
/hsdata/annotation/202007/15929070306-0-1-g.xml
/hsdata/annotation/202007/15930365679-5-0-c.xml
/hsdata/annotation/202007/15931117793-1-4-c.xml
/hsdata/annotation/202007/15919039146-5-

/hsdata/annotation/202007/15928652396-0-0-c.xml
/hsdata/annotation/202007/15919264713-0-1-g.xml
/hsdata/annotation/202007/15918709072-0-1-c.xml
/hsdata/annotation/202007/15929305756-1-1-c.xml
/hsdata/annotation/202007/15939936358-5-2-c.xml
/hsdata/annotation/202007/15930067817-5-2-c.xml
/hsdata/annotation/202007/15919040012-1-5-c.xml
/hsdata/annotation/202007/15929280022-5-1-c.xml
/hsdata/annotation/202007/15928606868-0-2-c.xml
/hsdata/annotation/202007/15916452701-0-1-c.xml
/hsdata/annotation/202007/15929342676-0-1-c.xml
/hsdata/annotation/202007/15938592648-0-0-c.xml
/hsdata/annotation/202007/15916257716-1-3-c.xml
/hsdata/annotation/202007/15930198277-5-2-c.xml
/hsdata/annotation/202007/15928485500-1-6-c.xml
/hsdata/annotation/202007/15916541556-1-0-c.xml
/hsdata/annotation/202007/15930365486-1-1-c.xml
/hsdata/annotation/202007/15930489406-0-1-g.xml
/hsdata/annotation/202007/15930066010-4-2-c.xml
/hsdata/annotation/202007/15919264654-5-1-g.xml
/hsdata/annotation/202007/15939706621-1-

/hsdata/annotation/202002/1585856157-0-1.xml
/hsdata/annotation/202002/1585318329_0_1.xml
/hsdata/annotation/202002/1585761648-1-3.xml
/hsdata/annotation/202002/1585168286_0_1.xml
/hsdata/annotation/202002/1585069767_0_0.xml
/hsdata/annotation/202002/1585922109-0-1.xml
/hsdata/annotation/202002/1585172133_0_1.xml
/hsdata/annotation/202002/1585936181-1-0.xml
/hsdata/annotation/202002/1585257086_0_1.xml
/hsdata/annotation/202002/1585865835-0-3.xml
/hsdata/annotation/202002/1585762649-0-0.xml
/hsdata/annotation/202002/1585834853-1-2.xml
/hsdata/annotation/202002/1585426279_0_0.xml
/hsdata/annotation/202002/1585947566-1-0.xml
/hsdata/annotation/202002/1585506163_0_0.xml
/hsdata/annotation/202002/1585505494_0_0.xml
/hsdata/annotation/202002/1585051802_0_0.xml
/hsdata/annotation/202002/1585069018_0_1.xml
/hsdata/annotation/202002/1585776071-1-2.xml
/hsdata/annotation/202002/1585078848_0_0.xml
/hsdata/annotation/202002/1585515141_0_3.xml
/hsdata/annotation/202002/1585257624_0_3.xml
/hsdata/an

...

/hsdata/annotation/202009/16002749752-1-5-c.xml
/hsdata/annotation/202009/16046097151-1-3-c.xml
/hsdata/annotation/202009/16062413754-1-3-c.xml
/hsdata/annotation/202009/16034914266-1-1-c.xml
/hsdata/annotation/202009/16069342883-1-3-c.xml
/hsdata/annotation/202009/15998619244-5-0-c.xml
/hsdata/annotation/202009/15995878342-5-2-c.xml
/hsdata/annotation/202009/16069381211-1-3-c.xml
/hsdata/annotation/202009/16002765688-1-6-c.xml
/hsdata/annotation/202009/16049497448-5-2-c.xml
/hsdata/annotation/202009/16035593550-7-3-c.xml
/hsdata/annotation/202009/16056530745-1-3-c.xml
/hsdata/annotation/202009/16058963851-5-2-c.xml
/hsdata/annotation/202009/15996854394-3-2-c.xml
/hsdata/annotation/202009/16043507116-5-0-c.xml
/hsdata/annotation/202009/16061482518-5-1-c.xml
/hsdata/annotation/202009/16063141037-5-1-c.xml
/hsdata/annotation/202009/16058112701-1-3-c.xml
/hsdata/annotation/202009/16057161192-1-3-c.xml
/hsdata/annotation/202009/16057091079-1-3-c.xml
/hsdata/annotation/202009/16055779773-1-

/hsdata/annotation/202009/16009651253-5-2-c.xml
/hsdata/annotation/202009/16059828825-1-3-c.xml
/hsdata/annotation/202009/16057281239-5-2-c.xml
/hsdata/annotation/202009/16063174601-1-3-c.xml
/hsdata/annotation/202009/16055383646-5-2-c.xml
/hsdata/annotation/202009/16035581998-7-0-c.xml
/hsdata/annotation/202009/16033131110-3-0-c.xml
/hsdata/annotation/202009/16046097212-1-2-c.xml
/hsdata/annotation/202009/16009703563-6-1-c.xml
/hsdata/annotation/202009/16058994507-1-3-c.xml
/hsdata/annotation/202009/16002723250-5-2-c.xml
/hsdata/annotation/202009/16035622194-7-0-c.xml
/hsdata/annotation/202009/16051026441-5-1-c.xml
/hsdata/annotation/202009/16010593557-5-1-c.xml
/hsdata/annotation/202009/16046783273-0-0-c.xml
/hsdata/annotation/202009/16043456587-1-1-c.xml
/hsdata/annotation/202009/16003599358-5-2-c.xml
/hsdata/annotation/202009/16055376068-5-2-c.xml
/hsdata/annotation/202009/15992551333-1-4-c.xml
/hsdata/annotation/202009/16033059538-7-2-c.xml
/hsdata/annotation/202009/16056225632-1-

/hsdata/annotation/202009/16008094536-1-2-c.xml
/hsdata/annotation/202009/16046944744-5-1-c.xml
/hsdata/annotation/202009/16035567485-7-0-c.xml
/hsdata/annotation/202009/16030377794-0-0-c.xml
/hsdata/annotation/202009/16056226033-1-3-c.xml
/hsdata/annotation/202009/16051167236-5-0-c.xml
/hsdata/annotation/202009/16063351119-1-3-c.xml
/hsdata/annotation/202009/16043542522-4-0-c.xml
/hsdata/annotation/202009/16058197378-1-3-c.xml
/hsdata/annotation/202009/16003673148-5-2-c.xml
/hsdata/annotation/202009/16028791393-5-2-c.xml
/hsdata/annotation/202009/16057096656-1-3-c.xml
/hsdata/annotation/202009/16055541292-1-2-c.xml
/hsdata/annotation/202009/16059877303-5-2-c.xml
/hsdata/annotation/202009/16046858349-1-3-c.xml
/hsdata/annotation/202009/16057102097-1-3-c.xml
/hsdata/annotation/202009/16062319112-1-3-c.xml
/hsdata/annotation/202009/16016688546-5-1-c.xml
/hsdata/annotation/202009/16030544700-5-0-c.xml
/hsdata/annotation/202009/15996994442-0-1-g.xml
/hsdata/annotation/202009/16052046742-1-

/hsdata/annotation/202001/1584962094_0_0.xml
/hsdata/annotation/202001/1584102916.xml
/hsdata/annotation/202001/1583685074.xml
/hsdata/annotation/202001/1583789100.xml
/hsdata/annotation/202001/1584572096_crop0.xml
/hsdata/annotation/202001/1583705731.xml
/hsdata/annotation/202001/1584101805.xml
/hsdata/annotation/202001/1583792591.xml
/hsdata/annotation/202001/1584820370_0_0.xml
/hsdata/annotation/202001/1584104229.xml
/hsdata/annotation/202001/1583789831.xml
/hsdata/annotation/202001/1583795492.xml
/hsdata/annotation/202001/1583795763.xml
/hsdata/annotation/202001/1584105967.xml
/hsdata/annotation/202001/1584820628_0_1.xml
/hsdata/annotation/202001/1584820559_0_2.xml
/hsdata/annotation/202001/1583794282.xml
/hsdata/annotation/202001/1584102938.xml
/hsdata/annotation/202001/1584555377_crop.xml
/hsdata/annotation/202001/1583795291.xml
/hsdata/annotation/202001/1583685429.xml
/hsdata/annotation/202001/1583790605.xml
/hsdata/annotation/202001/1583691521.xml
/hsdata/annotation/202001/1583

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
! aws s3 cp $tarball_name s3://jmduff.security-system/training_data/ --profile=jmduff

In [ ]:
tarball_name = DATA_DATE + "_jpeg_images.tar.gz"
jpeg_images_source = os.path.join(HSDATA, 'jpeg_images')
! tar cf - $jpeg_images_source | pigz > $tarball_name

In [ ]:
! aws s3 cp $tarball_name s3://jmduff.security-system/training_data/ --profile=jmduff